Временные ряды - финальный проект
====

Основное задание
---

В данном проекте главной задачей будет построить алгоритм, определяющий поиски аномалий.

Можно так же сделать симуляцию своей системы, то есть проверить, как система работала бы в продакшине, какие данные она бы использовала для перетренировки и как часто эта перетренировка бы соврешалась.

**На выбор** у вас два датасета. Один имеет много данных, а второй  временной ряд наоборот, содержат в себе еще очень мало информации. Вы так же можете использовать и другие датасеты из тех, что у вас есть в доступе на гитхабе. Но тогда вам прийдется сформулировать и ту задачу, которую вы пытаетесь решить, а именно пояснить, зачем в данном случае нам нужен поиск аномалий и как это будет применено на практике. 


Датасеты
------

Пассажиропоток нью-йоркского такси
--


In [1]:
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
# Use seaborn style defaults and set the default figure size
sns.set(rc={'figure.figsize':(20, 10)})
sns.set(font_scale=2)
sns.set_style("whitegrid")
sns.set_context("talk")
from pmdarima import auto_arima 
import pandas as pd 
import numpy as np
from statsmodels.tsa.stattools import adfuller

from sklearn.metrics import mean_absolute_error

from statsmodels.tsa.arima.model  import ARIMA


import statsmodels.api as sm

In [2]:

df = pd.read_csv('datatest/nyc_taxi.csv')
df.head()

,timestamp,value
0,2014-07-01 00:00:00,10844
1,2014-07-01 00:30:00,8127
2,2014-07-01 01:00:00,6210
3,2014-07-01 01:30:00,4656
4,2014-07-01 02:00:00,3820


In [3]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp',inplace=True)
df.index = pd.DatetimeIndex(df.index.values, freq=df.index.inferred_freq)


In [4]:
df = df.resample('H').sum()
df

,value
2014-07-01 00:00:00,18971
2014-07-01 01:00:00,10866
2014-07-01 02:00:00,6693
2014-07-01 03:00:00,4433
2014-07-01 04:00:00,4379
...,...
2015-01-31 19:00:00,56577
2015-01-31 20:00:00,48276
2015-01-31 21:00:00,48389
2015-01-31 22:00:00,53030


In [ ]:
result = adfuller(df['value'])
print('p-value: %f' % result[1])

Ряд стационарный - отличненько

In [5]:
size = int(len(df)*0.8)
train, test = df[0:size], df[size:]


In [ ]:
AutoARIMA_model = auto_arima(train["value"], start_p=1, start_q=1,
                         test='adf',
                         max_p=3, max_q=3, 
                         m=24, #12 is the frequncy of the cycle
                         start_P=0, 
                         seasonal=True, #set to seasonal
                         d=None, 
                         D=1, #order of the seasonal differencing
                         trace=False,
                         error_action='ignore',  
                         suppress_warnings=True, 
                         stepwise=True)
AutoARIMA_model.summary()

MemoryError: Unable to allocate 81.9 MiB for an array with shape (51, 51, 4129) and data type float64

Exception ignored in: 'statsmodels.tsa.statespace._kalman_smoother.dKalmanSmoother.reset_filter_method'
Traceback (most recent call last):
  File "statsmodels\tsa\statespace\_kalman_smoother.pyx", line 1014, in statsmodels.tsa.statespace._kalman_smoother.dKalmanSmoother.allocate_arrays


In [ ]:
order = (0,1,1)
seasonal_order = (0,1,[1,2],12)

SARIMA_model = ARIMA(train, order=order, seasonal_order=seasonal_order)

SARIMA_model_fit = SARIMA_model.fit()

predictions = SARIMA_model_fit.forecast(len(test))
Sarima_r2score = r2_score(test, predictions)
Sarima_mae = np.round(mean_absolute_error(test, predictions),2)
test.plot()
predictions.plot()
plt.show()